# 1. Client Mail Agents 📌

### **Que aprenderemos** 📚
1. 🧠 Utiliza Langchain agents.  
1. 🛠️ Usar el agente para programar y utilizar sus propias Tools.  
1. 💡 No requiere suscripciones, utiliza solamente modelos de LLM locales y ligeros, en este caso usamos `qwen3:7b` sobre `ollama`.

### **De qué trata el proyecto** 📚
Este proyecto realiza operaciones sobre los clientes de una empresa. 
En este caso, consulta los clientes de una empresa y genera un e-mail utilizando el idioma prefrido del cliente


### **Instalación de pre-requisitos** 🚀

Esta línea es una instrucción para instalar paquetes de software en tu entorno de programación de Python. Es similar a usar `pip install` en la terminal, pero el `%` al principio es una "magic command" que se usa en entornos interactivos como Jupyter Notebook o Google Colab.

**¿Cómo funciona?** 🧠

1. **`%pip`:** Invoca el instalador de paquetes de Python, `pip`. 🛠️
2. **`install`:** Le dice a `pip` que queremos instalar algo. 📦
3. **`-U --upgrade`:** Estas son opciones. `-U` es una forma abreviada de `--upgrade`. Juntas, le dicen a `pip` que actualice los paquetes a la última versión si ya están instalados. 🔧
4. **`pip`:** Este es el gestor de paquetes de Python. Es como una tienda donde puedes encontrar e instalar bibliotecas y herramientas creadas por otros programadores. 🛍️
5. **`langgraph langchain_ollama`:** Estos son los nombres de los paquetes específicos que queremos instalar. `pip` los buscará en el "Índice de Paquetes de Python" (PyPI), los descargará y los instalará en tu entorno. 📥

**Descripción de los paquetes:** 📚

- **`langchain`:** Es un framework que simplifica la creación de aplicaciones que utilizan modelos de lenguaje grandes (LLMs). Piensa en ello como un conjunto de herramientas para conectar LLMs con otras fuentes de datos y lógica. 🤖
- **`langchain_ollama`:** Una integración específica para usar modelos de lenguaje de la plataforma Ollama dentro de Langchain. Ollama te permite ejecutar LLMs localmente. 🧠
- **`dotenv`:** Permite cargar variables de entorno desde un archivo `.env`. Esto es útil para guardar información sensible (como claves de API) fuera de tu código. 🔒

In [ ]:
%%capture --no-stderr
%pip install -U pip langgraph langchain_ollama dotenv

### 📁 **Carga de dependencias**
```python
from dotenv import load_dotenv
import os
```
- 📌 **`from dotenv import load_dotenv`**: Importa la función `load_dotenv` de `dotenv`, que carga las variables de entorno desde un archivo `.env`.
- 📌 **`import os`**: Importa el módulo `os`, que permite interactuar con el sistema operativo (por ejemplo, acceder a variables de entorno).

 🔄 **Carga de variables de entorno**
```python
load_dotenv()
```
- 🚀 **`load_dotenv()`**: Carga las variables de entorno desde el archivo `.env` en el directorio actual. Esto es útil para gestionar claves API, rutas, etc., sin hardcodearlas en el código.

 📁 **Definición de carpeta de trabajo**
```python
workfolder = os.getenv('WORKFOLDER')
```
- 🗂️ **`os.getenv('WORKFOLDER')`**: Obtiene el valor de la variable de entorno `WORKFOLDER`, que probablemente define una carpeta donde se guardarán los archivos o datos del proyecto.

 🔄 **Configuración del modelo de lenguaje**
```python
llm_config = {"recursion_limit": 10}
```
- 🔄 **`llm_config`**: Define una configuración para el modelo de lenguaje.
  - 🧠 **`"recursion_limit": 20`**: Limita la profundidad de la recursión para evitar bucles infinitos.



In [9]:
from dotenv import load_dotenv
import os
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
load_dotenv()
workfolder = os.getenv('WORKFOLDER')
llm_config = {"recursion_limit": 10}

### 🧩 Bloque 1: Importaciones
```python
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent
```
- Se importan las librerías necesarias: `ChatOllama` para usar un modelo de lenguaje y `create_react_agent` para crear un agente de LangChain. 😊

### 🧠 Bloque 2: Crear el Agente
```python
simple_agent = create_react_agent(name="simple_agent", model=ChatOllama(model="qwen3"), tools=[])
```
- Se crea un agente llamado `simple_agent` usando el modelo `qwen3` de Ollama. No se le asignan herramientas (tools) por ahora. 😎

### 📄 Bloque 3: Mensaje de Entrada
```python
input_message = {"role": "user", "content": """
Generate the python code for a function that writes a string content in a file. 
a langchain tool with Annotated parameters and return type.
3 parameters: content, file_path and append (True or False)
should be able to handle exceptions and return an error message if the file cannot be written.
Function must have a 30 words docstring
use encoding utf8
give just the code, no introduction /no_think"""
}
```
- Se define un mensaje de entrada que simula una solicitud del usuario. El usuario pide que se genere un código de función en Python para escribir un contenido en un archivo, con anotaciones, manejo de excepciones y una documentación específica. 📝

### 🚀 Bloque 4: Usar el Agente
```python
for step in simple_agent.stream(
    {"messages": [input_message]}, llm_config, stream_mode="values"
):
    step["messages"][-1].pretty_print()
```
- Se usa el agente para procesar el mensaje de entrada. El resultado se muestra en pantalla usando `pretty_print()`. 🧠


In [ ]:
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent

simple_agent = create_react_agent(name="simple_agent", model=ChatOllama(model="qwen3"), tools=[])

input_message = {"role": "user", "content": """
Generate the python code for a function that writes a string content in a file. 
a langchain tool with Annotated parameters and return type.
3 parameters: content, file_path and append (True or False)
should be able to handle exceptions and return an error message if the file cannot be written.
Function must have a 30 words docstring
use encoding utf8
give just the code, no introduction /no_think"""
}

# Use the agent
for step in simple_agent.stream(
    {"messages": [input_message]}, llm_config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================


Generate the python code for a function that writes a string content in a file. 
a langchain tool with Annotated parameters and return type.
3 parameters: content, file_path and append (True or False)
should be able to handle exceptions and return an error message if the file cannot be written.
Function must have a 30 words docstring
use encoding utf8
give just the code, no introduction /no_think
================================== Ai Message ==================================
Name: simple_agent

<think>

</think>

```python
from langchain.tools import tool
from typing import Annotated

@tool
def write_to_file(
    content: Annotated[str, "The content to write to the file."],
    file_path: Annotated[str, "The path to the file."],
    append: Annotated[bool, "If True, append to the file. If False, overwrite."]
) -> Annotated[str, "Result message indicating success or error."]:
    """
    Writes content t

### 🔹 **Bloque 1: Importaciones**
### 🔹 **Bloque 2: Definición de la función `write_file`**
```python
@tool
def write_file(
    content: Annotated[str, "The content to be written to the file."],
    file_path: Annotated[str, "The path to the file where content will be written."],
    append: Annotated[bool, "If True, append to the file; if False, overwrite it."],
    encoding: Annotated[Optional[str], "The encoding of the file."] = 'utf-8'
) -> Annotated[str, "Result message indicating success or error."]:
```
- **¿Qué hace?** Define una función llamada `write_file` que es una herramienta (por el decorador `@tool`). 🛠️
- **¿Qué recibe?** 
  - `content`: El contenido a escribir. 📝
  - `file_path`: La ruta del archivo. 🗂️
  - `append`: Si es `True`, se añade al final; si es `False`, se sobrescribe. ✅
  - `encoding`: Codificación del archivo (por defecto `utf-8`). 🔐
- **¿Qué devuelve?** Un mensaje que indica si se escribió correctamente o hubo un error. 💬

### 🔹 **Bloque 3: Lógica de escritura del archivo**
```python
    """Writes content to a file in append or overwrite mode."""
    try:
        mode = 'a' if append else 'w'
        content = '\n'+content if append else content
        with open(file_path, mode, encoding=encoding) as f:
            f.write(content)
        return f"Content successfully written to {file_path}"
    except Exception as e:
        return f"Error writing to file: {str(e)}"
```
- **¿Qué hace?** 
  - **`mode = 'a' if append else 'w'`**: Define el modo de apertura del archivo (`'a'` para anexar, `'w'` para sobrescribir). 📁
  - **`content = '\n'+content if append else content`**: Si se está anexando, agrega una nueva línea al contenido. 📄
  - **`with open(...) as f`**: Abre el archivo con los parámetros definidos. 🚪
  - **`f.write(content)`**: Escribe el contenido en el archivo. ✍️
  - **`return ...`**: Devuelve un mensaje de éxito o error. 🚨
- **¿Por qué el `try-except`?** Para manejar errores en caso de que algo vaya mal, como que el archivo no exista o no se tenga permiso. ⚠️

### 🔹 **Bloque 4: Llamada a la función**
```python
write_file.invoke(input={'content':'this is a test\n', 'file_path':'/tmp4/file.txt', 'append': True})
```
- **¿Qué hace?** Llama a la función `write_file` con los parámetros especificados. 📡
- **¿Qué pasa?** 
  - Escribe el contenido `'this is a test\n'` en el archivo `/tmp4/file.txt`.
  - Como `append` es `True`, se añade al final del archivo. 📁
- **¿Dónde se guarda?** En la ruta `/tmp4/file.txt`. 🗂️


In [11]:
from langchain.tools import tool
from typing import Annotated, Optional

@tool
def write_file(
    content: Annotated[str, "The content to be written to the file."],
    file_path: Annotated[str, "The path to the file where content will be written."],
    append: Annotated[bool, "If True, append to the file; if False, overwrite it."],
    encoding: Annotated[Optional[str], "The encoding of the file."] = 'utf-8'
) -> Annotated[str, "Result message indicating success or error."]:
    """Writes content to a file in append or overwrite mode."""
    try:
        mode = 'a' if append else 'w'
        content = '\n'+content if append else content
        with open(file_path, mode, encoding=encoding) as f:
            f.write(content)
        return f"Content successfully written to {file_path}"
    except Exception as e:
        return f"Error writing to file: {str(e)}"

write_file.invoke(input={'content':'this is a test\n', 'file_path':'/tmp4/file.txt', 'append': True})

"Error writing to file: [Errno 2] No such file or directory: '/tmp4/file.txt'"

**Mensaje de entrada del usuario**  
```python src/1.client mail agents.ipynb
input_message = {"role": "user", "content": f"""
generate a csv list of 10 dummy clients with the following columns:
id, name, address, phone, email, prefered_languaje
write the list to file in '{workfolder}/clients.csv'
/no_think"""
}
```
📝 **Explicación:** Se define un mensaje de entrada del usuario que solicita generar una lista de 10 clientes ficticios en formato CSV, con columnas específicas, y guardar el archivo en una carpeta específica.  
🎯 *El usuario pide que se genere un archivo CSV con datos de clientes.*

**3. Uso del agente para procesar la solicitud**  
🤖 **Explicación:** Aquí se usa el agente para procesar el mensaje de entrada. El agente genera una respuesta, y se imprime el último mensaje de la conversación en un formato legible.  
📦 *El agente genera el CSV y lo escribe en el archivo especificado.*


In [ ]:
simple_agent = create_react_agent(name="simple_agent", model=ChatOllama(model="qwen3"), tools=[write_file])

input_message = {"role": "user", "content": f"""
generate a csv list of 10 dummy clients with the following columns:
id, name, address, phone, email, prefered_languaje
write the list to file in '{workfolder}/clients.csv'
/no_think"""
}

# Use the agent
for step in simple_agent.stream(
    {"messages": [input_message]}, llm_config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================


generate a csv list of 10 dummy clients with the following columns:
id, name, address, phone, email, prefered_languaje
write the list to file in '/home/daimler/workspaces/agents-course-huggingface/.workspace/clients.csv'
 /no_think
================================== Ai Message ==================================
Name: simple_agent

<think>

</think>
Tool Calls:
  write_file (6c85e30f-72f5-46a0-a79b-463b87e8b6e5)
 Call ID: 6c85e30f-72f5-46a0-a79b-463b87e8b6e5
  Args:
    append: False
    content: id,name,address,phone,email,prefered_languaje
1,John Doe,123 Main St,555-1234,john@example.com,English
2,Jane Smith,456 Oak Ave,555-5678,jane@example.com,Spanish
3,Michael Johnson,789 Pine Rd,555-9012,michael@example.com,French
4,Emily Davis,321 Maple Ln,555-3456,emily@example.com,German
5,David Wilson,654 Cedar Blvd,555-7890,david@example.com,Italian
6,Sarah Lee,987 Oak St,555-2345,sarah@example.com,Dutch
7,Jame

### 1. **Creación del código de la segunda Tool** 😊  
```python
input_message = {"role": "user", "content": f"""
Generate the python code for a function that return dict with all the client details for a given id. 
read clients from csv file_path in utf8
is a langchain tool with typing Annotated params.
params: id: int, file_path: str, encoding: Optional[str] = 'utf-8'
handle exceptions
Function must have a 20 word docstring
give just the code, no introduction /no_think"""
}
```
- **¿Qué hace?** Define un mensaje de entrada que simula una petición del usuario para generar código.  
- **Detalles:** Contiene instrucciones sobre cómo debe ser la función, incluyendo parámetros, manejo de excepciones y una docstring específica.  


In [24]:
input_message = {"role": "user", "content": f"""
Generate the python code for a function that return dict with all the client details for a given id. 
read clients from csv file_path in utf8
is a langchain tool with typing Annotated params.
params: id: int, file_path: str, encoding: Optional[str] = 'utf-8'
handle exceptions
Function must have a 20 word docstring
give just the code, no introduction /no_think"""
}

# Use the agent
for step in simple_agent.stream({"messages": [input_message]}, llm_config, stream_mode="values"):
    step["messages"][-1].pretty_print()


================================ Human Message =================================


Generate the python code for a function that return dict with all the client details for a given id. 
read clients from csv file_path in utf8
is a langchain tool with typing Annotated params.
params: id: int, file_path: str, encoding: Optional[str] = 'utf-8'
handle exceptions
Function must have a 20 word docstring
give just the code, no introduction /no_think
================================== Ai Message ==================================
Name: simple_agent

<think>

</think>

from typing import Dict, Annotated, Optional
from langchain.tools import tool
import csv

@tool
def get_client_details(
    id: Annotated[int, "Client ID to retrieve details for"],
    file_path: Annotated[str, "Path to the CSV file containing client data"],
    encoding: Optional[Annotated[str, "File encoding, default is 'utf-8'"]] = 'utf-8'
) -> Dict[str, any]:
    """Retrieve client details by ID from a CSV file. Handles excepti

### 📁 **Definición de la función `get_client_details`**  
La función se llama `get_client_details` y recibe tres parámetros:  
- `id`: El ID del cliente que queremos buscar.  
- `file_path`: La ruta del archivo CSV donde están los datos de los clientes.  
- `encoding`: La codificación del archivo (por defecto es `'utf-8'`).  

La función devuelve un diccionario con los detalles del cliente o un error si algo salió mal.

### 🧠 **Bloque `try`**  
Este bloque intenta leer el archivo CSV y buscar el cliente por su ID.  
```python
with open(file_path, mode='r', encoding=encoding) as file:
    reader = csv.DictReader(file)
    for row in reader:
        if int(row['id']) == id:
            return dict(row)
    return {}
```
- Abre el archivo en modo lectura (`'r'`).  
- Usa `csv.DictReader` para leer el archivo como un diccionario.  
- Recorre cada fila del CSV y compara el ID del cliente con el `id` proporcionado.  
- Si encuentra un cliente con el mismo ID, devuelve su información como un diccionario.  
- Si no lo encuentra, devuelve un diccionario vacío `{}`.

### ⚠️ **Bloque `except`**  
Si ocurre un error durante la lectura del archivo (por ejemplo, el archivo no existe o tiene un formato incorrecto), se captura y se devuelve un mensaje de error.
```python
except Exception as e:
    return {"error": str(e)}
```
- Captura cualquier excepción y la convierte en un mensaje de error.  
- Devuelve un diccionario con la clave `"error"` y el mensaje de error.


In [38]:
from langchain_core.tools import tool 
from typing import Dict, Annotated, Optional
import csv

@tool
def get_client_details(
    id: Annotated[int, "Client ID to retrieve details for"],
    file_path: Annotated[str, "Path to the CSV file containing client data"],
    encoding: Optional[Annotated[str, "File encoding, default is 'utf-8'"]] = 'utf-8'
) -> Dict[str, any]:
    """Retrieve client details by ID"""
    try:
        with open(file_path, mode='r', encoding=encoding) as file:
            reader = csv.DictReader(file)
            for row in reader:
                if int(row['id']) == id:
                    return dict(row)
            return {}
    except Exception as e:
        return {"error": str(e)}


get_client_details.invoke(input={'id':6, 'file_path': f'{workfolder}/clients.csv'})

{'id': '6',
 'name': 'Sarah Lee',
 'address': '987 Oak St',
 'phone': '555-2345',
 'email': 'sarah@example.com',
 'prefered_languaje': 'Dutch'}

**Bloque 1: Crear un mensaje de entrada**  
```python
input_message = {"role": "user", "content": f"""
get the client details of a client with id '{random.randint(0, 10)}'
file is in {workfolder}/clients.csv
generate an email explaining we could not deliver the package at his address nor contact him at his phone number.
use they prefered language
especify the actual address and phone number of the client.
"""
}
```
- **¿Qué hace?** Crea un mensaje de entrada en formato JSON que simula una consulta de un usuario.  
- **Detalles:**  
  - Pide los detalles de un cliente con un ID aleatorio entre 0 y 10. 😊  
  - Indica que los datos están en un archivo CSV llamado `clients.csv` en la carpeta `workfolder`. 🗂️  
  - Solicita generar un email explicando que no se pudo entregar el paquete ni contactar al cliente. 📧  
  - Menciona usar el idioma preferido del cliente. 🌍  
  - Pide especificar la dirección y el número de teléfono del cliente. 📞


In [29]:
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent
import random

simple_agent = create_react_agent(name="simple_agent", model=ChatOllama(model="qwen3"), tools=[get_client_details])

input_message = {"role": "user", "content": f"""
get the client details of a client with id '{random.randint(0, 10)}'
file is in {workfolder}/clients.csv
generate an email explaining we could not deliver the package at his address nor contact him at his phone number.
use they prefered language
especify the actual address and phone number of the client.
/no_think"""
}

# Use the agent
for step in simple_agent.stream(
    {"messages": [input_message]}, llm_config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================


get the client details of a client with id '5'
file is in /home/daimler/workspaces/agents-course-huggingface/.workspace/clients.csv
generate an email explaining we could not deliver the package at his address nor contact him at his phone number.
use they prefered language
especify the actual address and phone number of the client.
/no_think
================================== Ai Message ==================================
Name: simple_agent

<think>

</think>
Tool Calls:
  get_client_details (626a5861-9f6a-465d-9609-8085e7cbac3e)
 Call ID: 626a5861-9f6a-465d-9609-8085e7cbac3e
  Args:
    file_path: /home/daimler/workspaces/agents-course-huggingface/.workspace/clients.csv
    id: 5
================================= Tool Message =================================
Name: get_client_details

{"id": "5", "name": "David Wilson", "address": "654 Cedar Blvd", "phone": "555-7890", "email": "david@example.com", "prefe